<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

<div class="alert alert-block alert-danger">
<b>Important:</b> This exercise requires the completion of <a href="./02.query_with_ezpresto.ipynb" <b>Exercise 2:</b> Connect and Query Data Sources with EzPresto</a></div>

# **Exercise 3:** Explore Retail Data with Apache Spark

This exercise will introduce **Apache Spark on HPE AI Essentials**. We'll leverage Spark's powerful distributed processing capabilities to analyze and fix the sales information.

In this exercise, you will:

- Set up a Spark session for interacting with data.
- Generate sample sales data for different countries and currencies.
- Explore techniques for data loading, transformation, and analysis using Spark SQL and DataFrames.
- Create Delta Tables and perform version control.

Feel free to modify and extend the code examples to suit your specific data analysis needs.

Let's get started!

### **Prerequisites:**

As instructed in the [Introductory notebook](./00.introduction.ipynb), ensure that you have run `pip install -r requirements.txt` in a Terminal window, located in the same working directory, prior to running this notebook. 

<div class="alert alert-block alert-danger">
    <b>Important:</b> Make sure you selected <b>PySpark</b> for your notebook kernel - check the top right corner!
</div>

## **1. Create Spark Session**

Think about the most recent Excel spreadsheet you edited. It probably had tens or even hundreds of rows across tens of columns. When you run an Excel command, such as a *SUM()* or a *VLOOKUP()*, you may have noticed that it took a far bit of time to process. Maybe, even the fans of your laptop sped up a bit as your computer worked to crunch the numbers. 

Now, scale that same command out to a spreadsheet with tens of **millions** of rows across **thousands** of columns. That is the Big Data that companies must work with on a daily basis, and no single PC is going to run any *VLOOKUP* command on data of that size.

Instead of spreadsheets, the enterprise world is largely built upon **tables** in a variety of formats. To query these tables to retrieve certain data takes a **mammoth** amount of compute. It makes no sense to have a single **compute server** executing these queries - it would be far faster to parallelize queries across several computers. Enter **Apache Spark**.

### Introduction to Apache Spark on HPE AI Essentials

Apache Spark is a popular open-source big data framework that **distributes the computations** required to perform queries on large sets of data. This distribution, along with working with data in-memory rather than directly from storage disks, drastically brings down the time usually taken to query and index data. The combination of speed, versatility, and ease of use made Spark the go-to framework when working with big data. 

Apache Spark comes pre-installed with **HPE AI Essentials** and can leverage as much or as little of the compute available in a AIE cluster as a user desired. The core components of an Apache Spark deployment include:

<img src="./images/exercise1/spark_archi.PNG" alt="Drawing" style="width: 60%;"/>

**Driver:** The driver program coordinates the execution of Spark jobs. It submits tasks to executors, schedules operations, and manages communication between various components.

**Workers:** These are machines in the Spark cluster that manage executors. Each worker runs one or more executors. When running Spark on a HPE AI Essentials deployment, Spark Workers are Kubernetes pods distributed among worker nodes of the AIE cluster, allowing them to scale across multiple machines as required. 

**Executors:** Executors reside on worker nodes and carry out the actual computations assigned by the driver program. They partition and distribute the workload across machines in the cluster.

**JVM:**  Spark utilizes the Java Virtual Machine (JVM) on each worker node to execute executors.

On **HPE AI Essentials**, you will use Apache Spark to analyze large datasets at high speed with a unified platform for batch processing, streaming, and machine learning.

### Create a Spark Interactive Session

Let's begin using Spark! Here, you use HPE AI Essentials' native integration of **Apache Livy** to create and manage an interactive Spark session. Livy is an open-source REST service that enables remote and interactive analytics on Apache Spark clusters. It provides a way to interact with Spark clusters programmatically using a REST API, allowing you to submit Spark jobs, run interactive queries, and manage Sparksessions from web applications without the need for a specific Spark client. As a result, multiple AIE users can interact with your Spark cluster concurrently and reliably!

First, let's connect to the Livy endpoint and create a new Spark interactive session. The Spark interactive
session is particularly useful for exploratory data analysis, prototyping, and iterative development. It allows you to
interactively work with large datasets, perform transformations, apply analytical operations, and build ML models using
Spark's distributed computing capabilities. 

To communicate with Livy and manage your sessions you use Sparkmagic, an open-source tool that provides a Jupyter kernel
extension. Sparkmagic integrates with Livy, to provide the underlying communication layer between the Jupyter kernel and
the Spark cluster.

**Execute the cell below**, then:

1. Select the `Add Endpoint` tab.
1. Select `Single Sign-on` and ensure there is a Livy address in the `Address` field. 
1. Click `Add Endpoint`.
1. Select the `Create Session` tab.
1. Provide a name (e.g. `retail-demo`).
1. Select `python` under the Language field.
1. Click `Create Session` (right side).

The session will take a few minutes for your session to initialize. 

Once ready, the Manage Sessions pane will activate, displaying
your session ID. When the session state turns to idle, you're all set!

In [ ]:
%manage_spark

Now, let's check the status of the session.

1. Navigate back to the AIE dashboard.
1. In the sidebar navigation menu, select `Spark Interactive Sessions`.

![image.png](./images/exercise1/menu.PNG)

3. Here, you can check the status of your session. It will take 2-3 minutes to start. When the `State` says `Idle`, the session is ready. 

![image.png](./images/exercise1/session.PNG)

4. Scroll back up to the Notebook cell of the session (%manage_spark command). Confirm under the `Manage Sessions` tab that the session should now be visible as `Idle` too. 

![image.png](./images/exercise1/session2.PNG)

<div class="alert alert-block alert-danger">
    <b>Important:</b> Set your <b>Username</b>, your <b>Domain</b> and the name of your <b>Presto connection</b> (catalog) here !
</div>

In [ ]:
USERNAME=""
DOMAIN=""
CATALOG=f"retail{USERNAME}"

Next, let's import the required libraries for working with Spark in this notebook.

In [ ]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from delta.tables import DeltaTable
import os

You can now instantiate the Spark session. We'll add delta extensions to the configuration to be able to interact with the delta tables.

In [ ]:
spark = SparkSession.builder \
    .appName("RetailDataPipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Configuration
delta_path = f"file:///mounts/shared-volume/shared/{USERNAME}/delta-tables/"
os.makedirs(delta_path, exist_ok=True)

## **2. Create Delta Tables**

In this section, we will create Delta Tables from our SQL database that we can query using EzPresto. Delta Tables are a type of table that can be created in Delta Lake, which is an extension of Apache Parquet file format.

### Define an ETL Pipeline to create Delta Tables 

First, let's define some functions that will:

1. Load the data in from a SQL database and and save it as a Delta table

In [ ]:
# List of tables to extract
tables = [
    "source_catalog",
    "source_customers",
    "source_orders",
    "source_order_products",
    "source_stock"
]

SCHEMA="public"

def extract_and_save_table(table_name):
    """Extract a single table from Presto and save to Delta"""
    try:
        print(f"Processing table: {table_name}")
        
        # Presto connection configuration
        uri = f"jdbc:presto://ezpresto.{DOMAIN}:443/{CATALOG}/{SCHEMA}"
        query = f"SELECT * FROM {CATALOG}.{SCHEMA}.{table_name}"
        
        # Read from Presto
        df = spark.read.format("jdbc") \
            .option("driver", "com.facebook.presto.jdbc.PrestoDriver") \
            .option("url", uri) \
            .option("SSL", "true") \
            .option("IgnoreSSLChecks", "true") \
            .option("query", query) \
            .load()
        
        # Write to Delta format
        df.write.format("delta") \
            .mode("overwrite") \
            .save(f"{delta_path}{table_name}")
            
        print(f"Successfully saved {table_name} to Delta format")
        return True
        
    except Exception as e:
        print(f"Error processing table {table_name}: {str(e)}")
        return False

# Process all tables
for table in tables:
    success = extract_and_save_table(table)
    if not success:
        print(f"Failed to process table {table}")

print("All tables processed")

In [ ]:
# Check which tables exist
existing_tables = []
for table in tables:
    try:
        df = spark.read.format("delta").load(f"{delta_path}{table}")
        existing_tables.append(table)
        print(f"Found Delta table: {table}")
    except Exception as e:
        print(f"Could not load {table}: {str(e)}")

if not existing_tables:
    print("\nERROR: No Delta tables found. Please confirm:")
    print(f"1. The path is correct: {delta_path}")
    print("2. The tables were created successfully")
    print("3. You have read permissions to the location")
else:
    print("\nTables available:", existing_tables)

2. Clean the data, in this case by checking the spelling and dropping the `null` values. Then we save it back to the same Delta table

In [ ]:
from pyspark.sql.functions import col, trim, when, lit

# Clean product names and categories
cleaned_catalog = spark.read.format("delta").load(f"{delta_path}source_catalog") \
    .withColumn("product_name", trim(col("product_name"))) \
    .withColumn("product_category", 
        when(col("product_category") == "Toyz", "Toys")
        .when(col("product_category") == "Clothng", "Clothing")
        .when(col("product_category") == "Eletronics", "Electronics")
        .otherwise(col("product_category"))) \
    .filter(col("product_id").isNotNull()) \
    .filter(col("price_cents") > 0)  # Remove negative prices

cleaned_catalog.write.format("delta") \
    .mode("overwrite") \
    .save(f"{delta_path}source_catalog")

Let's compare the result, but first we need to define the function

In [ ]:
def compare_table(table_name):
    delta_table = f"{delta_path}{table_name}"
    
    # Read current version (latest)
    df_current = spark.read.format("delta").load(delta_table)
    
    # Find the latest version number
    latest_version = DeltaTable.forPath(spark, delta_table).history(1).collect()[0].version
    
    # Read previous version (latest - 1)
    df_previous = spark.read.format("delta").option("versionAsOf", latest_version - 1).load(delta_table)
    
    # Show data
    print(f"Current Version of {table_name}:")
    df_current.show()
    
    print(f"Previous Version {table_name}:")
    df_previous.show()

In [ ]:
compare_table("source_catalog")

In [ ]:
# Clean customer data
cleaned_customers = spark.read.format("delta").load(f"{delta_path}source_customers") \
    .withColumn("customer_name", trim(col("customer_name"))) \
    .withColumn("customer_surname", trim(col("customer_surname"))) \
    .withColumn("customer_email",
        when(
            (col("customer_email").contains("@")) & 
            (col("customer_email").contains(".")),
            col("customer_email")
        ).otherwise(lit(None))) \
    .filter(col("customer_id").isNotNull())

cleaned_customers.write.format("delta") \
    .mode("overwrite") \
    .save(f"{delta_path}source_customers")

In [ ]:
compare_table("source_customers")

In [ ]:
from pyspark.sql.functions import current_date

# Clean stock data
cleaned_stock = spark.read.format("delta").load(f"{delta_path}source_stock") \
    .filter(col("product_quantity") > 0) \
    .filter(col("entry_date") <= lit(current_date())) \
    .filter(col("product_id").isNotNull()) \
    .filter(col("purchase_price_cents") > 0)

cleaned_stock.write.format("delta") \
    .mode("overwrite") \
    .save(f"{delta_path}source_stock")

In [ ]:
compare_table("source_stock")

In [ ]:
# Clean orders data
cleaned_orders = spark.read.format("delta").load(f"{delta_path}source_orders") \
    .filter(col("order_date") <= current_date()) \
    .filter(col("customer_id").isNotNull()) \
    .withColumn("order_status",
        when(col("order_status").isin(["completed", "pending", "cancelled", "shipped"]),
            col("order_status")
        ).otherwise(lit("pending")))

cleaned_orders.write.format("delta") \
    .mode("overwrite") \
    .save(f"{delta_path}source_orders")

In [ ]:
compare_table("source_orders")

In [ ]:
# Clean order products data
cleaned_order_products = spark.read.format("delta").load(f"{delta_path}source_order_products") \
    .filter(col("product_quantity") > 0) \
    .filter(col("order_id").isNotNull()) \
    .filter(col("product_id").isNotNull())

cleaned_order_products.write.format("delta") \
    .mode("overwrite") \
    .save(f"{delta_path}source_order_products")

In [ ]:
compare_table("source_order_products")

Great! We've just created functions that will **extract** the data from our SQL tables, **transform** them to get rid of inconsistencies, then **load** them as Delta Table into a new directory.

You guessed it! We have just created an **ETL pipeline!** 

## **3. Connecting the Data Source**

### Connect Delta Tables as Data Source using Apache Hive.

Let's take those Delta Tables you just and make them available to other applications in AI Essentials by connecting them as an **Data Source**. Apache Hive gives an SQL-like interface to query data stored in various databases and file systems, like the delta tables you created. 

1. Navigate back to the AI Essentials dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under the `Structured Data` tab, click `Add New Data Source`.
1. Under **Hive**, click `Create Connection`.

- **Name**: `deltavince`
- **Hive Metastore:** `discovery`
- **Data Dir:** `file:/data/shared/vince/delta-tables/`
- **File Type:** `PARQUET`
5. Click `Connect`.



In [ ]:
def display_dt_connection_details():
    """Display database connection details without connecting"""
    connection_details = {
        "Name": f"delta{USERNAME}",
        "Hive Metastore": "deltadiscovery",
        "Data Dir": f"{delta_path}"
    }
    
    print("\nDelta Table Connection Details:")
    print("=" * 40)
    
    max_key_length = max(len(key) for key in connection_details.keys())
    
    for key, value in connection_details.items():
        print(f"{key.title():>{max_key_length}} : {value}")
    
    print("=" * 40 + "\n")

display_dt_connection_details()

# **Conclusion**

In this exercise, you learned to perform the basics of data engineering - all within a single notebook! 

**HPE AI Essentials** makes this possible by natively supporting and including the most widely used open-source data tools and frameworks and making them readily available out-of-the-box, such that you spent this time performing invaluable data preperation for upcoming exercises instead of hours installing and connecting them all!

In the next exercise, you will learn how to use this data to build your first AI Agent

Next: <a href="./04.create_AI_agent.ipynb" style="color: black"><b style="color: #01a982;">Exercise 4:</b> Creating an AI Agent to analyze the data</a>

Previous: <a href="./02.query_with_ezpresto.ipynb" style="color: black"><b style="color: #01a982;">Exercise 2:</b> Connect and Query Data Sources with EzPresto</a>
